In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

We're ready to build our first neural network. We will have multiple features we feed into our model, each of which will go through a set of perceptron models to arrive at a response which will be trained to our output.

Like many models we've covered, this can be used as both a regression or classification model.

First, we need to load our dataset. For this example we'll use The Museum of Modern Art in New York's [public dataset](https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv) on their collection.

In [3]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [4]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.

In [5]:
# select columns
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# convert urls to booleans
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# drop films and some other tricky rows
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# drop missing data
artworks = artworks.dropna()

In [5]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a Model

Now, let's see if we can use multi-layer perceptron modeling (or "MLP") to see if we can classify the department a piece should go into using everything but the department name.

Before we import MLP from SKLearn and establish the model we first have to ensure correct typing for our data and do some other cleaning.

In [6]:
# get data types
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

The `DateAcquired` column is an object. Let's transform that to a datetime object and add a feature for just the year the artwork was acquired.

In [6]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

In [7]:
# remove multiple nationalities, genders, and artists
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_person\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# convert dates to start date, cutting down number of distinct examples
artworks['Date'] = pd.Series(artworks.Date.str.extract('([0-9]{4})', expand=False))[:-1]

In [8]:
# Final column drops and NA drop
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'],1)

# create dummies separately
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, except for artists co'z it can slows program down
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [10]:
# # import neural network
# from sklearn.neural_network import MLPClassifier

# # establish and fit model with a single 1000 perceptron layer
# mlp = MLPClassifier(hidden_layer_sizes=(1000,))
# mlp.fit(X,Y)

In [11]:
# mlp.score(X,Y)

In [12]:
# Y.value_counts()/len(Y)

In [13]:
# from sklearn.model_selection import cross_val_score
# cross_val_score(mlp,X,Y,cv=5)

Now we got a lot of information from all of this. Firstly we can see that the model seems to overfit, though there is still so remaining performance when validated with cross validation. This is a feature of neural networks that aren't given enough data for the number of features present. _Neural networks, in general, like_ a lot _of data_. You may also have noticed something also about neural networks: _they can take a_ long _time to run_. Try increasing the layer size by adding a zero. Feel free to interrupt the kernel if you don't have time...

Also note that we created bools for artist's name but left them out. Both of the above points are the reason for that. It would take much longer to run and it would be much more prone to overfitting.

## Model parameters

Now, before we move on and let you loose with some tasks to work on the model, let's go over the parameters.

We included one parameter: hidden layer size. Remember in the previous lesson, when we talked about layers in a neural network. This tells us how many and how big to make our layers. Pass in a tuple that specifies each layer's size. Our network is 1000 neurons wide and one layer. (100, 4, ) would create a network with two layers, one 100 wide and the other 4.

How many layers to include is determined by two things: computational resources and cross validation searching for convergence. It's generally less than the number of input variables you have.

You can also set an alpha. Neural networks like this use a regularization parameter that penalizes large coefficients just like we discussed in the advanced regression section. Alpha scales that penalty.

Lastly, we'll discuss the activation function. The activation function determines whether the output from an individual perceptron is binary or continuous. By default this is a 'relu', or 'rectified linear unit function' function. In the exercise we went through earlier we used this binary function, but we discussed the _sigmoid_ as a reasonable alternative. The _sigmoid_ (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexity.

If you want to learn more about these, study [activation functions](https://en.wikipedia.org/wiki/Activation_function) and [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron). The [Deep Learning](http://www.deeplearningbook.org/) book referenced earlier goes into great detail on the linear algebra involved.

You could also just test the models with cross validation. Unless neural networks are your specialty cross validation should be sufficient.

For the other parameters and their defaults, check out the [MLPClassifier documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

### Single 1000 perceptron layer

In [15]:
# # import neural network
# from sklearn.neural_network import MLPClassifier

# # establish and fit model with a single 1000 perceptron layer
# mlp = MLPClassifier(hidden_layer_sizes=(1000,))
# mlp.fit(X_train,y_train)
# mlp.score(X_train,y_train)

In [16]:
# cross_val_score(mlp, X_train, y_train, cv=5)

### Four 250 perceptron layers 

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
mlp = MLPClassifier(hidden_layer_sizes=(250,4))
mlp.fit(X_train,y_train)
print(mlp.score(X_train,y_train))
print(cross_val_score(mlp,X_train,y_train,cv=5))

0.6227227820588817
[0.62265393 0.62269494 0.62273595 0.62277697 0.62275212]


In [20]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
mlp = MLPClassifier(hidden_layer_sizes=(100,250))
mlp.fit(X_train,y_train)
print(mlp.score(X_train,y_train))
print(cross_val_score(mlp,X_train,y_train,cv=5))

0.8266877428703154


C:\Users\vungu\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\vungu\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\vungu\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[0.81771485 0.81119526 0.80400421 0.819116   0.81943347]


In [14]:
mlp = MLPClassifier(hidden_layer_sizes=(500,300,200))
mlp.fit(X_train,y_train)
print(mlp.score(X_train,y_train))
print(cross_val_score(mlp,X_train,y_train,cv=5))

C:\Users\vungu\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.8235790028321148


C:\Users\vungu\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[0.81198551 0.80433351 0.80642824 0.79574496 0.80047428]


mlp = MLPClassifier(hidden_layer_sizes=(200,100,100))
mlp.fit(X_train,y_train)
print(mlp.score(X_train,y_train))
print(cross_val_score(mlp,X_train,y_train,cv=5))

### Four 250 perceptron layers with alpha

In [12]:
mlp = MLPClassifier(hidden_layer_sizes=(250,4), alpha=1e-03)
mlp.fit(X_train,y_train)
print(mlp.score(X_train,y_train))
print(cross_val_score(mlp,X_train,y_train,cv=5))

0.6227227820588817
[0.62265393 0.62269494 0.62273595 0.62277697 0.62275212]


### Four 250 perceptron layers with alpha and learning rate

In [13]:
mlp = MLPClassifier(hidden_layer_sizes=(250,4), alpha=1e-03, learning_rate='adaptive')
mlp.fit(X_train,y_train)
print(mlp.score(X_train,y_train))
print(cross_val_score(mlp,X_train,y_train,cv=5))

0.6227227820588817
[0.74646032 0.62269494 0.62273595 0.62277697 0.62275212]


* Increasing the numbers of perceptron layers while adding alpha and learning rate don't change the accuracy score.
* Keeping the numbers of perceptron layers low while adjusting the numbers of perceptron can increase accuracy.  